<a href="https://colab.research.google.com/github/adarshakumar395/Dijkstra-Algorithm-Implementation/blob/main/Dijkstra_Algorithm_with_UI_and_Animations_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Required libraries

In [ ]:
!pip install numpy matplotlib ipywidgets ipython

Implementation of Dijkshtra'ss Algorithm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.animation as animation
import heapq

# Define the Node class
class Node:
    def __init__(self, position, cost=0):
        self.position = position
        self.cost = cost
        self.parent = None

    def __lt__(self, other):
        return self.cost < other.cost

def create_grid(bounds, resolution):
    x = np.arange(bounds[0][0], bounds[0][1] + resolution, resolution)
    y = np.arange(bounds[1][0], bounds[1][1] + resolution, resolution)
    z = np.arange(bounds[2][0], bounds[2][1] + resolution, resolution)
    return np.meshgrid(x, y, z, indexing='ij')

def get_neighbors(node, resolution, bounds):
    directions = [
        (resolution, 0, 0), (-resolution, 0, 0),
        (0, resolution, 0), (0, -resolution, 0),
        (0, 0, resolution), (0, 0, -resolution),
        (resolution, resolution, 0), (resolution, -resolution, 0),
        (-resolution, resolution, 0), (-resolution, -resolution, 0),
        (0, resolution, resolution), (0, resolution, -resolution),
        (0, -resolution, resolution), (0, -resolution, -resolution),
        (resolution, 0, resolution), (resolution, 0, -resolution),
        (-resolution, 0, resolution), (-resolution, 0, -resolution)
    ]

    neighbors = []
    for direction in directions:
        neighbor_pos = node.position + np.array(direction)
        if all(bounds[i][0] <= neighbor_pos[i] <= bounds[i][1] for i in range(3)):
            neighbors.append(Node(neighbor_pos))
    return neighbors

def is_collision_free(node, obstacles, resolution):
    for obstacle in obstacles:
        if np.linalg.norm(obstacle - node.position) < resolution:
            return False
    return True

def dijkstra(start, goal, obstacles, bounds, resolution=1):
    start_node = Node(np.array(start))
    goal_node = Node(np.array(goal))

    open_list = []
    heapq.heappush(open_list, start_node)

    closed_list = set()
    closed_list.add(tuple(start_node.position))

    while open_list:
        current_node = heapq.heappop(open_list)

        if np.array_equal(current_node.position, goal_node.position):
            return reconstruct_path(current_node)

        neighbors = get_neighbors(current_node, resolution, bounds)

        for neighbor in neighbors:
            if not is_collision_free(neighbor, obstacles, resolution):
                continue

            neighbor.cost = current_node.cost + np.linalg.norm(neighbor.position - current_node.position)
            neighbor.parent = current_node

            if tuple(neighbor.position) not in closed_list:
                heapq.heappush(open_list, neighbor)
                closed_list.add(tuple(neighbor.position))

    return None

def reconstruct_path(node):
    path = []
    while node is not None:
        path.append(node.position)
        node = node.parent
    return path[::-1]

def plot_path(path, obstacles, start, goal):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot start and goal
    ax.scatter(*start, color='green', s=100, label='Start')
    ax.scatter(*goal, color='red', s=100, label='Goal')

    # Plot obstacles
    for obstacle in obstacles:
        ax.scatter(*obstacle, color='blue', s=100)

    # Plot path
    if path:
        path = np.array(path)
        ax.plot(path[:,0], path[:,1], path[:,2], color='black', linewidth=2, label='Path')

    ax.set_xlim(bounds[0])
    ax.set_ylim(bounds[1])
    ax.set_zlim(bounds[2])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    plt.show()

def animate_path(path, obstacles, start, goal):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot start and goal
    ax.scatter(*start, color='green', s=100, label='Start')
    ax.scatter(*goal, color='red', s=100, label='Goal')

    # Plot obstacles
    for obstacle in obstacles:
        ax.scatter(*obstacle, color='blue', s=100)

    # Plot path
    path = np.array(path)
    line, = ax.plot(path[:,0], path[:,1], path[:,2], color='black', linewidth=2, label='Path')

    # Drone marker
    drone, = ax.plot([], [], [], marker='o', markersize=10, color='orange', label='Drone')
    drone.set_data(path[0, 0], path[0, 1])
    drone.set_3d_properties(path[0, 2])

    # Animation function
    def update(num, path, line, drone):
        line.set_data(path[:num, :2].T)
        line.set_3d_properties(path[:num, 2])
        drone.set_data(path[num, 0], path[num, 1])
        drone.set_3d_properties(path[num, 2])
        return line, drone,

    anim = animation.FuncAnimation(fig, update, len(path), fargs=[path, line, drone], interval=100, blit=False)

    ax.set_xlim(bounds[0])
    ax.set_ylim(bounds[1])
    ax.set_zlim(bounds[2])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()

    # Display animation in Colab
    display(HTML(anim.to_html5_video()))

# Interactive UI elements for user input
start_x = widgets.FloatText(value=10.0, description='Start X:')
start_y = widgets.FloatText(value=19.0, description='Start Y:')
start_z = widgets.FloatText(value=25.0, description='Start Z:')
goal_x = widgets.FloatText(value=34.0, description='Goal X:')
goal_y = widgets.FloatText(value=33.0, description='Goal Y:')
goal_z = widgets.FloatText(value=27.0, description='Goal Z:')
obstacles_input = widgets.Text(value='5,5,5; 7,7,7; 3,8,6', description='Obstacles:')
run_button = widgets.Button(description='Run Dijkstra')

ui = widgets.VBox([start_x, start_y, start_z, goal_x, goal_y, goal_z, obstacles_input, run_button])
display(ui)

def run_dijkstra(b):
    start = np.array([start_x.value, start_y.value, start_z.value])
    goal = np.array([goal_x.value, goal_y.value, goal_z.value])
    obstacles = []
    for obs in obstacles_input.value.split(';'):
        try:
            obstacle = np.array(list(map(float, obs.split(','))))
            if len(obstacle) == 3:
                obstacles.append(obstacle)
        except ValueError:
            print(f"Invalid obstacle entry: {obs}")

    path = dijkstra(start, goal, obstacles, bounds, resolution=1)

    if path:
        print("Path found:")
        for point in path:
            print(point)
        plot_path(path, obstacles, start, goal)
        animate_path(path, obstacles, start, goal)
    else:
        print("Path not found.")

run_button.on_click(run_dijkstra)

# Parameters for the Dijkstra's algorithm
bounds = [(0, 100), (0, 100), (0, 100)]
